<a href="https://colab.research.google.com/github/noemigarcia27/SImulacion-II/blob/main/Eficiencia_de_metodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Eficiencia del Método de Monte Carlo**

Las librerias que vamos a ocupar son las siguientes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time

Nuestra función para este ejercicio es:
$$ g(x) = \sqrt{arctan(x)} dx  $$

In [3]:
def g(x):
  return np.sqrt(np.arctan(x))

**Monte Carlo: acierto y error**

Este es el codigo para una unica estimacion del método de acierto y error

In [4]:
def estimacion(N):
  aciertos=0
  for i in range(N):  #se generan pares de números aleatorios
    x = rd.random()
    y = rd.random()
    if y <= g(x):  #ve cuales puntos quedaron debajo de la curva de la función
      aciertos += 1 #los aciertos se van acumulando
  I=aciertos/N  #Fórmula del estimador p^ = N_a / N
  return I

In [5]:
estimacion(1000)

0.598

Bucle para hacer el muestreo

In [6]:
def mc_acierto(N,n):  #n es el numero de estimaciones
  start_time = time.perf_counter()  #empieza el contador para el tiempo en el que se tarda
  lista=[]
  for i in range(N):
    E=estimacion(n)
    lista.append(E)
  end_time = time.perf_counter()
  execution_time = end_time - start_time
  return np.mean(lista), np.var(lista), execution_time

**Monte Carlo crudo**

In [7]:
def mc_crudo(N,n):
  start_time = time.perf_counter()
  resultados_c=[] #se guardan las estimaciones
  for i in range(n): #n numero de estimaciones, comienza a hacer el bucle
    lista_c=[]
    for j in range(N):  #N es el tamaño de la muestra
      u=rd.random()
      lista_c.append(g(u))
    resultados_c.append(lista_c)
  end_time = time.perf_counter()
  execution_time = end_time - start_time
  return np.mean(lista_c), np.var(lista_c), execution_time

Para revisar que método es más eficiente ocupamos la fórmula
$$ \epsilon = \frac{t_1 \text{Var}(\theta_1)}{t_2 \text{Var}(\theta_2)} $$


In [8]:
mean1, var1, t1 = mc_acierto(1000,100)
mean2, var2, t2 = mc_crudo(1000,100)

Si $ \epsilon = \frac{t_1 \text{Var}(\theta_1)}{t_2 \text{Var}(\theta_2)} < 1 $, entones el primer método es más eficiente, de lo contrario el segundo será más eficiente

In [9]:
E = (t2 * var2) / (t1 * var1)

In [10]:
print("Método Acierto y Error:  media:", mean1, " varianza:", var1, " tiempo:", t1)
print("Método Crudo:            media:", mean2, " varianza:", var2, " tiempo:", t2)
print("Eficiencia (E):", E)

Método Acierto y Error:  media: 0.63047  varianza: 0.0023644791000000005  tiempo: 0.1675755180000067
Método Crudo:            media: 0.6187053532797698  varianza: 0.044920149576207276  tiempo: 0.16754752800000006
Eficiencia (E): 18.994731905568862


####Reducción de la varianza para el método de Monte Carlo crudo
Vamos a hacer la reducción con el método de variables antipéticas, usando el teorema
$$ \text{Var} ( \frac{ X_1 + X_2 }{2} ) = \frac{1}{4} [ \text{Var} (X_1) + \text{Var} (X_2) + 2 \text{Cov} ( X_1 + X_2 )] $$

En el siguiente codigo en vez de usar muestras u (como en el Método MC crudo) se utiliza 1-u

In [11]:
def mc_crudo_g_1u(N, n):
  start_time=time.perf_counter()
  resultados_c=[]
  for i in range(n): # Genera N muestras pero evalúa en (1-u) en lugar de u
    lista_c=[]
    for j in range(N):
      u=rd.random()
      lista_c.append(g(1-u))
    resultados_c.append(lista_c)
  end_time=time.perf_counter()
  execution_time=end_time-start_time
  return np.mean(lista_c), np.var(lista_c), execution_time

En el siguiente lo que hacermos es usar ambos y sacamos el promedio es decir, es lo que hace realmente la reduccion de la varianza
$$ \text{Var} ( \frac{ f(u) + f(1-u) }{2} ) < \text{Var} (f(u)) $$

In [12]:
def mc_crudo_reduccion_var(N,n):
  start_time=time.perf_counter()
  resultados_c=[]
  for i in range(n):
    lista_c=[]
    for j in range(N):
      u=rd.random()
      lista_c.append((g(u) + g(1-u))/2.0) #promedio g(u) y g(1-u)
    resultados_c.append(lista_c)
  end_time=time.perf_counter()
  execution_time=end_time-start_time
  return np.mean(lista_c), np.var(lista_c), execution_time

In [13]:
mean_c, var_c, t_c = mc_crudo(1000, 30)
mean_as, var_as, t_as = mc_crudo_g_1u(1000, 30)
mean_ac, var_ac, t_ac = mc_crudo_reduccion_var(1000, 30)

In [14]:
print("Monte Carlo crudo con u:               media =", mean_c, " varianza =", var_c, " tiempo =", t_c)
print("Monte Carlo crudo con 1-u:             media =", mean_as, " varianza =", var_as, " tiempo =", t_as)
print("Monte Carlo crudo con la red de var:   media =", mean_ac, " varianza =", var_ac, " tiempo =", t_ac)

Monte Carlo crudo con u:               media = 0.6251987702365354  varianza = 0.044259280476903154  tiempo = 0.06259057799999823
Monte Carlo crudo con 1-u:             media = 0.6340276191994  varianza = 0.041693464134693256  tiempo = 0.04992815700001074
Monte Carlo crudo con la red de var:   media = 0.6292724954608073  varianza = 0.002698715256366613  tiempo = 0.10233163199998785


Para ver en cuanto se redujo la varianza ocupamos lo siguiente
$$ \text{Reduccion} = \frac{\text{Varianza Original - Varianza Nueva}}{\text{Varianza Original}} * 100 $$

In [15]:
reduccion=((var_c-var_ac)/var_c)*100

In [16]:
reduccion

np.float64(93.90248728111396)

Vemos que se reduce la varianza en 93.90%

---



###Reduccion de varianza para Método acierto y error
Podemos hacerlo con antiteticas con pares, es decir $ (u,v) $ y $ (1-u, 1-v) $  

Esto funciona porque los dos indicaadores estan negativamente correlacionados
$$ \text{Var}(\tilde{I}) = \frac{ \text{Var}(\tilde{Z}) }{ N/2 } = \frac{1}{N/2} \cdot \frac{1}{4} [ 2 \text{Var}(Z) + 2 \text{Cov}(Z_1,Z_2) ] $$

Si $ \text{Cov}(Z_1,Z_2) < 0 $, entonces

$$Var(\tilde{I})<Var(\hat{I})$$

donde $ \tilde{I} = \frac{1}{N/2} \sum_{k=1}^{N/2} \tilde{Z}_k, \hat{I} = \frac{1}{N} \sum_{i=1}^{N} Z_i, \tilde{Z} = \frac{Z_1 + Z_2}{2} $

In [17]:
def mc_acierto_reduccion_var(N, n): #n numero de estimaciones
    start_time=time.perf_counter()
    resultados=[]
    for i in range(n):  # n
        aciertos=0
        for j in range(N // 2):  # usamos N/2 porque son pares
            u1, v1 = rd.random(), rd.random()
            u2, v2 = 1 - u1, 1 - v1
            aciertos += int(v1 <= g(u1))  #incrementa aciertos
            aciertos += int(v2 <= g(u2))
        I = aciertos / N
        resultados.append(I)
    end_time = time.perf_counter()
    execution_time = (end_time - start_time) / n
    return np.mean(resultados), np.var(resultados), execution_time

In [18]:
mean, var, t = mc_acierto_reduccion_var(1000, 30)

In [19]:
print("Método Acierto y Error:                  media:", mean1, " varianza:", var1, " tiempo:", t1)
print("Método acierto y error con red de var:   media =", mean, " varianza =", var, " tiempo =", t)

Método Acierto y Error:                  media: 0.63047  varianza: 0.0023644791000000005  tiempo: 0.1675755180000067
Método acierto y error con red de var:   media = 0.6312333333333332  varianza = 8.831222222222237e-05  tiempo = 0.002050842300000018


Para ver en cuanto se redujo la varianza ocupamos lo siguiente
$$ \text{Reduccion} = \frac{\text{Varianza Original - Varianza Nueva}}{\text{Varianza Original}} * 100 $$

In [20]:
reduccion=((var1-var)/var1)*100

In [21]:
reduccion

np.float64(96.26504534456564)

La varianza se redujo en un 96.25%